In [1]:
import logging
import time

# Configure logger
logger: logging.Logger = logging.getLogger("run_workflow_logger")
logger.setLevel(logging.INFO)
logger.propagate = False  # Prevent duplicate logs from parent loggers

# Set formatter
formatter: logging.Formatter = logging.Formatter(
    fmt="%(asctime)s - %(levelname)s - %(message)s",
    datefmt="%Y-%m-%d %H:%M:%S"
)

# Configure and attach stream handler
stream_handler: logging.StreamHandler = logging.StreamHandler()
stream_handler.setFormatter(formatter)
logger.addHandler(stream_handler)

In [ ]:
import datetime
import os
import uuid
import base64
import logging
import sys
import time
from typing import Dict, Any, List
import pandas as pd
from langchain_community.document_loaders import PyPDFLoader
from langchain_community.llms import LlamaCpp
from langchain_core.callbacks import CallbackManager, StreamingStdOutCallbackHandler
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_huggingface import HuggingFaceEmbeddings, HuggingFacePipeline, HuggingFaceEndpoint
from langchain_community.vectorstores import Chroma
from langchain.prompts import ChatPromptTemplate
from langchain.schema import StrOutputParser
from langchain.schema.runnable import RunnablePassthrough, RunnableLambda, RunnableMap
from langchain.schema.document import Document
from transformers import AutoModelForCausalLM, AutoTokenizer, pipeline

# Add the src directory to the path to import base_service
sys.path.append(os.path.abspath(os.path.join(os.path.dirname(__file__), "../..")))
from core.chatbot_service.chatbot_service import ChatbotService
from src.utils import (
    get_context_window, 
    dynamic_retriever, 
    format_docs_with_adaptive_context,
    login_huggingface,
    load_config_and_secrets,
    clean_code
)
from src.prompt_templates import format_rag_chatbot_prompt

2025-07-03 22:24:53 - INFO - Notebook execution started.


In [3]:
# === Standard Library Imports ===
import os
import sys
import logging
import warnings
from datetime import datetime
from pathlib import Path
from typing import List

# === Third-Party Imports ===
import mlflow
import torch
from langchain_community.document_loaders import PyMuPDFLoader
from langchain_community.llms import LlamaCpp
from langchain_core.callbacks import CallbackManager, StreamingStdOutCallbackHandler
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_huggingface import HuggingFaceEmbeddings, HuggingFacePipeline, HuggingFaceEndpoint
from langchain_community.vectorstores import Chroma
from langchain.prompts import ChatPromptTemplate
from langchain.schema import StrOutputParser
from langchain.schema.runnable import RunnablePassthrough, RunnableLambda, RunnableMap
from langchain.schema.document import Document
from transformers import AutoModelForCausalLM, AutoTokenizer, pipeline
import promptquality as pq
import galileo_protect as gp
from galileo_protect import ProtectTool, ProtectParser, Ruleset

# Define the relative path to the 'src' directory (one level up from current working directory)
sys.path.append(os.path.abspath(os.path.join(os.getcwd(), "..")))

# === Project-Specific Imports (from src.utils) ===
from src.utils import (
    load_config_and_secrets,
    configure_proxy,
    initialize_llm,
    setup_galileo_environment,
    initialize_galileo_evaluator,
    initialize_galileo_protect,
    initialize_galileo_observer,
    configure_hf_cache,
)
from src.prompt_templates import format_rag_chatbot_prompt

# === Import the service class ===
from core.chatbot_service.chatbot_service import ChatbotService

/opt/conda/lib/python3.12/site-packages/mlflow/pyfunc/utils/data_validation.py:168: UserWarning: Add type hints to the `predict` method to enable data validation and automatic signature inference during model logging. Check https://mlflow.org/docs/latest/model/python_model.html#type-hint-usage-in-pythonmodel for more details.
  color_warning(
/opt/conda/lib/python3.12/site-packages/mlflow/pyfunc/utils/data_validation.py:168: UserWarning: Add type hints to the `predict` method to enable data validation and automatic signature inference during model logging. Check https://mlflow.org/docs/latest/model/python_model.html#type-hint-usage-in-pythonmodel for more details.
  color_warning(


In [4]:
# Configuration parameters for the RAG chatbot
MODEL_SOURCE = "local"  # Can be: "local", "hugging-face-local", "hugging-face-cloud"
CHATBOT_SERVICE_NAME = "VanillaRAGChatbot"

# Load configuration and secrets
config = load_config("../configs/config.yaml")
secrets = load_secrets("../configs/secrets.yaml")

print("✅ Configuration loaded successfully")

# Model Service 

In this section, we demonstrate how to deploy a RAG-based chatbot service with integrated Galileo Protect and Observe capabilities. This service provides a REST API endpoint that allows users to query the knowledge base with natural language questions, upload new documents to the knowledge base, and manage conversation history, all with built-in safeguards against sensitive information and toxicity. This service encapsulates all the functionality we developed in this notebook, including the document retrieval system, RAG-based question answering capabilities, and Galileo integration for protection, observation and evaluation. It demonstrates how to use our ChatbotService from the src/service directory. 

In [5]:
mlflow.set_tracking_uri('/phoenix/mlflow')
# === Set MLflow experiment context ===
mlflow.set_experiment(MLFLOW_EXPERIMENT_NAME)

# === Validate local model file path ===
if not os.path.exists(LOCAL_MODEL_PATH):
    logger.info(f"⚠️ Warning: Model file not found at {LOCAL_MODEL_PATH}. Please verify the path.")

# === Log and register model to MLflow ===
with mlflow.start_run(run_name=MLFLOW_RUN_NAME) as run:
    
    # Log model artifacts using custom ChatbotService
    ChatbotService.log_model(
        artifact_path=MLFLOW_MODEL_NAME,
        config_path=CONFIG_PATH,
        secrets_path=SECRETS_PATH,
        docs_path=DATA_PATH,
        model_path=LOCAL_MODEL_PATH,
        demo_folder=DEMO_FOLDER
    )

    # Construct the URI for the logged model
    model_uri = f"runs:/{run.info.run_id}/{MLFLOW_MODEL_NAME}"

    # Register the model into MLflow Model Registry
    mlflow.register_model(
        model_uri=model_uri,
        name=MLFLOW_MODEL_NAME
    )

    logger.info(f"✅ Model registered successfully with run ID: {run.info.run_id}")

2025-07-03 22:25:02,250 - INFO - PyTorch version 2.7.0 available.
2025-07-03 22:25:02,550 - INFO - Use pytorch device_name: cuda
2025-07-03 22:25:02,550 - INFO - Load pretrained SentenceTransformer: sentence-transformers/all-mpnet-base-v2
/opt/conda/lib/python3.12/site-packages/mlflow/pyfunc/__init__.py:3212: UserWarning: An input example was not provided when logging the model. To ensure the model signature functions correctly, specify the `input_example` parameter. See https://mlflow.org/docs/latest/model/signatures.html#model-input-example for more details about the benefits of using input_example.
  color_warning(


2025/07/03 22:27:11 WARNING mlflow.utils.requirements_utils: Detected one or more mismatches between the model's dependencies and the current Python environment:
 - PyPDF (current: uninstalled, required: PyPDF)
To fix the mismatches, call `mlflow.pyfunc.get_model_dependencies(model_uri)` to fetch the model's environment and install dependencies using the resulting environment file.
2025-07-03 22:29:18,470 - INFO - Model and artifacts successfully registered in MLflow.
Registered model 'AIStudio-Chatbot-Model' already exists. Creating a new version of this model...
Created version '2' of model 'AIStudio-Chatbot-Model'.
2025-07-03 22:29:19 - INFO - ✅ Model registered successfully with run ID: 589b398f5bb548979a98ec29430fc1c5


In [6]:
end_time: float = time.time()
elapsed_time: float = end_time - start_time
elapsed_minutes: int = int(elapsed_time // 60)
elapsed_seconds: float = elapsed_time % 60

logger.info(f"⏱️ Total execution time: {elapsed_minutes}m {elapsed_seconds:.2f}s")
logger.info("✅ Notebook execution completed successfully.")

2025-07-03 22:29:19 - INFO - ⏱️ Total execution time: 4m 25.90s
2025-07-03 22:29:19 - INFO - ✅ Notebook execution completed successfully.


Built with ❤️ using [**HP AI Studio**](https://hp.com/ai-studio).